In [ ]:
import requests
import json
from bs4 import BeautifulSoup

In [ ]:
webpageurl = 'https://www.uow.edu.au/student/central/academicconsideration/UOW053903.html'
data = requests.get(webpageurl) # Get a UOW page.

In [ ]:
data.text[:1000]

In [ ]:
soup = BeautifulSoup(data.text,'html.parser') 

In [ ]:
soup.prettify()[:1000] # Prettify() used to make html readable.

In [ ]:
soup.head # Test getting the head of the document.

In [ ]:
soup.body #test body of html document

In [ ]:
page_title = soup.title.text # get the title of this page whilst stripping html characters.
print(page_title)

In [ ]:
page_content = soup.find('div', {'id' : 'content'})

In [ ]:
content = page_content.text
print(content)

In [ ]:
rest = content.split("Last reviewed", 1)[0]
print(rest)

In [ ]:
import os
current_directory = os.getcwd() 
if current_directory.split("/")[-1] != "recommender-keywords":
    os.chdir("../recommender-keywords")
    print(os.getcwd())

/Users/rorys/PycharmProjects/sailas-rs/recommender-keywords


In [20]:
# Load the sitemap scrape.
path = "uow-scrape.json"
sitemap_file = open(path, 'r')
sitemap = sitemap_file.read()

In [ ]:
pages = json.loads(sitemap)

# Test getting the first URL
print(list(pages[0].values())[0])

In [ ]:
def send(data):
    # Variable to store our query in.
    query = {}
    
    print(v['id'], v['question'], v['answer'])
    query['id'] = v['id']
    query['question'] = v['question']
    query['answer'] = v['answer']
    
    # Payload is now ready.
    payload = json.dumps(query)
    r = requests.post("http://HOST:9200/sailas/resource/", data=payload, headers=headers)
    print(r.status_code, r.reason, v)

In [ ]:
# This will iterate through the sitemap scrape and get each page's content and title.
# Data will be sent to a SQLite3 DB.

import sqlite3
conn = sqlite3.connect('keywords.db')

c = conn.cursor()
count_pages = 0

try:
    c.execute("CREATE TABLE content_scrape (title text, url text, content text)")
except:
    print("Already created.")

for page in pages:
    webpageurl = list(page.values())[0]
    if "/student/" in webpageurl:
        try:
            data = requests.get(webpageurl) # Get a UOW page.
        except:
            print("Request timed out.", webpageurl)

        soup = BeautifulSoup(data.text, 'html.parser')

        try:
            page_title = soup.title.text # Get the title of this page whilst stripping html characters.
            print(page_title)
        except:
            print("This page has no title.")
            page_title = ""
        try:
            page_content = soup.find('div', {'id' : 'content'})
            content = page_content.text
            rest = content.split("Last reviewed", 1)[0].rstrip('\n')

        except:
            print("Empty page.")
            rest = ""

        query = []
        query.append([str(page_title), str(webpageurl), str(rest)])

        try:
            c.executemany('INSERT INTO content_scrape VALUES (?, ?, ?)', query)
            conn.commit()
        except:
            print("DB write failed.")

        count_pages = count_pages + 1
        print(count_pages, "have been completed.")
    else:
        print("Skipping", webpageurl)

conn.close()
    

In [ ]:
if "/student/" in list(page.values())[0]: 
   print('success')